# Training

In [21]:
import pandas as pd
import numpy as np
import os
import random

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

import importlib
import train_models_util
import feature_importance
importlib.reload(train_models_util)
importlib.reload(feature_importance)
from train_models_util import (
    train_xgb_optuna, 
    train_catboost_optuna, 
    train_lgbm_optuna,
    train_best_base_models_from_mlflow,
    train_ensemble, 
    train_neural_network_ensemble,
    train_tree_optuna,
    minimize_eval_metric_with_threshold
)
from evaluate_models_util import evaluate_and_log
from feature_importance import select_important_features, get_top_features_shap
from sklearn.model_selection import train_test_split

# set all random seeds for reproducibility
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [2]:
EXPERIMENT_NAME = "Fraud-detection-custom-thresholds_v2"

In [3]:
data_folder = 'ieee-fraud-detection-data/processed/'

train = pd.read_csv(f'{data_folder}train_processed.csv')
test = pd.read_csv(f'{data_folder}test_processed.csv')
target = pd.read_csv(f'{data_folder}target.csv')['isFraud']

In [4]:
cat_cols = np.array(train.select_dtypes(include=['object']).columns)
for c in cat_cols:
    if c in train.columns:
        if train[c].isnull().sum()>0 or test[c].isnull().sum()>0:
            train[c].fillna('missing', inplace=True)
            test[c].fillna('missing', inplace=True)
        train[c] = train[c].astype('category')
        test[c] = test[c].astype('category')

## 0. Create tuning, training , validation, and test sets 

In [6]:
# get all fraud samples
fraud_indices = target[target == 1].index

# sample 5× non-fraud, without replacement
non_fraud_indices = target[target == 0].sample(
    n=len(fraud_indices) * 3,
    random_state=42
).index

# build tuning dataset
X_tune = pd.concat([train.loc[fraud_indices], train.loc[non_fraud_indices]], axis=0)
y_tune = pd.concat([target.loc[fraud_indices], target.loc[non_fraud_indices]], axis=0)

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=0.2, stratify=target, random_state=42)

In [9]:
X_train.shape, X_valid.shape, X_tune.shape

((472432, 398), (118108, 398), (82652, 398))

In [10]:
del train, target, fraud_indices, non_fraud_indices, cat_cols, c
import gc
gc.collect()

21

## 1. Full Features

### 1.1. XGBoost

In [27]:
model_xgboost_full, best_params_xgboost_full, hist_df, plot_paths = train_xgb_optuna(X_train, y_train, X_tune, y_tune, X_valid, y_valid, early_stopping_rounds=10)

[I 2025-11-30 11:52:51,503] A new study created in memory with name: xgboost_aucpr_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-30 11:54:10,902] Trial 0 finished with value: 0.9438730674951549 and parameters: {'learning_rate': 0.19105628024524834, 'max_depth': 11, 'subsample': 0.6656717060834552, 'colsample_bytree': 0.6447858685514625, 'min_child_weight': 0.04822036256991612, 'gamma': 1.346076192732022}. Best is trial 0 with value: 0.9438730674951549.
[I 2025-11-30 11:57:12,887] Trial 1 finished with value: 0.9483650900704378 and parameters: {'learning_rate': 0.049469965577439115, 'max_depth': 12, 'subsample': 0.7897089283395412, 'colsample_bytree': 0.8860897411723251, 'min_child_weight': 0.013579128280901189, 'gamma': 1.8938526849680586}. Best is trial 1 with value: 0.9483650900704378.
[I 2025-11-30 11:59:48,272] Trial 2 finished with value: 0.9364268743984301 and parameters: {'learning_rate': 0.025994831802352212, 'max_depth': 10, 'subsample': 0.8869131377110315, 'colsample_bytree': 0.8160569661595771, 'min_child_weight': 0.011656132067452092, 'gamma': 3.8530385006234042}. Best is trial 1 with valu

[INFO] Best XGBoost params: {'learning_rate': 0.047977461787374734, 'max_depth': 14, 'subsample': 0.7449890839436284, 'colsample_bytree': 0.70110582598937, 'min_child_weight': 0.0010226677164813366, 'gamma': 0.14922602556210918}


In [28]:
evaluate_and_log(model_xgboost_full, X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.978296886263691), 'pr_auc': np.float64(0.9170423426563677), 'precision': 0.9559659090909091, 'recall': 0.8141785627873216, 'f1': 0.8793937018162812, 'custom_loss': np.float64(0.6515646696244115)}
[INFO] Logged SHAP summary plot.


2025/11/30 13:42:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.978296886263691),
 'pr_auc': np.float64(0.9170423426563677),
 'precision': 0.9559659090909091,
 'recall': 0.8141785627873216,
 'f1': 0.8793937018162812,
 'custom_loss': np.float64(0.6515646696244115)}

In [29]:
evaluate_and_log(model_xgboost_full,  X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.978296886263691), 'pr_auc': np.float64(0.9170423426563677), 'precision': 0.9217231415812976, 'recall': 0.8490200822646987, 'f1': 0.8838790931989925, 'custom_loss': np.float64(0.5308531174856911)}
[INFO] Logged SHAP summary plot.


2025/11/30 13:44:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.978296886263691),
 'pr_auc': np.float64(0.9170423426563677),
 'precision': 0.9217231415812976,
 'recall': 0.8490200822646987,
 'f1': 0.8838790931989925,
 'custom_loss': np.float64(0.5308531174856911)}

In [30]:
evaluate_and_log(model_xgboost_full, X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)

[INFO] Logged metrics: {'roc_auc': np.float64(0.978296886263691), 'pr_auc': np.float64(0.9170423426563677), 'precision': 0.8589378852536748, 'recall': 0.8766029518509557, 'f1': 0.867680517303317, 'custom_loss': np.float64(0.4368459376164189)}
[INFO] Logged SHAP summary plot.


2025/11/30 13:45:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.978296886263691),
 'pr_auc': np.float64(0.9170423426563677),
 'precision': 0.8589378852536748,
 'recall': 0.8766029518509557,
 'f1': 0.867680517303317,
 'custom_loss': np.float64(0.4368459376164189)}

In [31]:
threshOpt, minScore = minimize_eval_metric_with_threshold(model_xgboost_full,  X_valid, y_valid)
print(f"Optimal threshold: {threshOpt}, Minimum eval metric score: {minScore}")
evaluate_and_log(model_xgboost_full,  X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=threshOpt)

Optimal threshold: 0.0011392603628337383, Minimum eval metric score: 0.2979815084498933
[INFO] Logged metrics: {'roc_auc': np.float64(0.978296886263691), 'pr_auc': np.float64(0.9170423426563677), 'precision': 0.1986487849147928, 'recall': 0.9533026857004597, 'f1': 0.32878541327658867, 'custom_loss': np.float64(0.2979815084498933)}
[INFO] Logged SHAP summary plot.


2025/11/30 13:46:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.978296886263691),
 'pr_auc': np.float64(0.9170423426563677),
 'precision': 0.1986487849147928,
 'recall': 0.9533026857004597,
 'f1': 0.32878541327658867,
 'custom_loss': np.float64(0.2979815084498933)}

### 1.2. LightGBM

In [41]:
model_lgbm_full, best_params_lgbm_full, hist_df, plot_paths = train_lgbm_optuna(X_train, y_train, X_tune, y_tune, X_valid, y_valid)

[I 2025-11-30 18:24:17,873] A new study created in memory with name: lgbm_aucpr_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-30 18:25:15,547] Trial 0 finished with value: 0.9354406354964823 and parameters: {'num_leaves': 89, 'max_depth': 19, 'learning_rate': 0.04038013995157655, 'feature_fraction': 0.7595537716395071, 'bagging_fraction': 0.7586046530394261, 'bagging_freq': 5, 'min_child_samples': 52, 'lambda_l1': 4.510556069643136, 'lambda_l2': 7.351593732709463e-07}. Best is trial 0 with value: 0.9354406354964823.
[I 2025-11-30 18:25:54,953] Trial 1 finished with value: 0.9382900451228401 and parameters: {'num_leaves': 98, 'max_depth': 20, 'learning_rate': 0.29315441288699473, 'feature_fraction': 0.9077682600481172, 'bagging_fraction': 0.6814818706169963, 'bagging_freq': 9, 'min_child_samples': 47, 'lambda_l1': 1.711599919982529e-05, 'lambda_l2': 4.193220145208493e-07}. Best is trial 1 with value: 0.9382900451228401.
[I 2025-11-30 18:26:59,548] Trial 2 finished with value: 0.9507117129689933 and parameters: {'num_leaves': 166, 'max_depth': 16, 'learning_rate': 0.09504356911475484, 'feature_fracti

[INFO] Best LightGBM params: {'num_leaves': 227, 'max_depth': 17, 'learning_rate': 0.07656990174288049, 'feature_fraction': 0.936270507330223, 'bagging_fraction': 0.942671114143771, 'bagging_freq': 9, 'min_child_samples': 56, 'lambda_l1': 2.954976945576723e-05, 'lambda_l2': 9.299566730170337e-07}


In [42]:
model_lgbm_full

LGBMClassifier(bagging_fraction=0.942671114143771, bagging_freq=9,
               categorical_feature=['ProductCD', 'card4', 'card6',
                                    'P_emaildomain', 'R_emaildomain', 'M1',
                                    'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8',
                                    'M9', 'id_12', 'id_15', 'id_16', 'id_23',
                                    'id_27', 'id_28', 'id_29', 'id_30', 'id_31',
                                    'id_33', 'id_34', 'id_35', 'id_36', 'id_37',
                                    'id_38', 'DeviceType', ...],
               device='cpu', eval_metric='aucpr',
               feature_fraction=0.936270507330223,
               lambda_l1=2.954976945576723e-05, lambda_l2=9.299566730170337e-07,
               learning_rate=0.07656990174288049, max_depth=17,
               min_child_samples=56, n_estimators=1000, num_leaves=227,
               objective='binary', random_state=42,
               scale_pos_weight=27.580278281911674, verbosity=-1)

In [43]:
evaluate_and_log(model_lgbm_full,  X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9056815767165312), 'pr_auc': np.float64(0.4363099973978413), 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'custom_loss': np.float64(3.49933958749619)}
[INFO] Logged SHAP summary plot.


2025/11/30 19:05:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9056815767165312),
 'pr_auc': np.float64(0.4363099973978413),
 'precision': 0.0,
 'recall': 0.0,
 'f1': 0.0,
 'custom_loss': np.float64(3.49933958749619)}

In [44]:
evaluate_and_log(model_lgbm_full, X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9056815767165312), 'pr_auc': np.float64(0.4363099973978413), 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'custom_loss': np.float64(3.49933958749619)}
[INFO] Logged SHAP summary plot.


2025/11/30 19:05:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9056815767165312),
 'pr_auc': np.float64(0.4363099973978413),
 'precision': 0.0,
 'recall': 0.0,
 'f1': 0.0,
 'custom_loss': np.float64(3.49933958749619)}

In [45]:
evaluate_and_log(model_lgbm_full, X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9056815767165312), 'pr_auc': np.float64(0.4363099973978413), 'precision': 0.18336463643681244, 'recall': 0.8267602225985966, 'f1': 0.30015811665495434, 'custom_loss': np.float64(0.735072984048498)}
[INFO] Logged SHAP summary plot.


2025/11/30 19:05:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9056815767165312),
 'pr_auc': np.float64(0.4363099973978413),
 'precision': 0.18336463643681244,
 'recall': 0.8267602225985966,
 'f1': 0.30015811665495434,
 'custom_loss': np.float64(0.735072984048498)}

In [46]:
threshOpt, minScore = minimize_eval_metric_with_threshold(model_lgbm_full, X_valid, y_valid)
print(f"Optimal threshold: {threshOpt}, Minimum eval metric score: {minScore}")
evaluate_and_log(
    model_lgbm_full, 
    X_valid, y_valid, 
    experiment_name=EXPERIMENT_NAME, 
    run_name="LightGBM_Optuna_fullfeatures", 
    hp_search_history=hist_df, hp_search_plots=plot_paths, 
    prediction_threshold=threshOpt)

Optimal threshold: 0.04439166260234292, Minimum eval metric score: 0.6371795305991127
[INFO] Logged metrics: {'roc_auc': np.float64(0.9056815767165312), 'pr_auc': np.float64(0.4363099973978413), 'precision': 0.08162434372656166, 'recall': 0.9216065811759013, 'f1': 0.14996653411551636, 'custom_loss': np.float64(0.6371795305991127)}
[INFO] Logged SHAP summary plot.


2025/11/30 19:05:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9056815767165312),
 'pr_auc': np.float64(0.4363099973978413),
 'precision': 0.08162434372656166,
 'recall': 0.9216065811759013,
 'f1': 0.14996653411551636,
 'custom_loss': np.float64(0.6371795305991127)}

### 1.3. CatBoost

In [ ]:
model_catboost_full, best_params_catboost_full, hist_df, plot_paths = train_catboost_optuna(X_train, y_train, X_valid, y_valid, X_valid, y_valid, early_stopping_rounds=10)

[I 2025-11-29 20:48:33,506] A new study created in memory with name: catboost_aucpr_optimization


[Optuna CatBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-29 20:53:43,096] Trial 0 finished with value: 0.9212252621609099 and parameters: {'learning_rate': 0.2860429973310247, 'depth': 6, 'l2_leaf_reg': 9.775721614705137, 'subsample': 0.8397188154264389, 'border_count': 72}. Best is trial 0 with value: 0.9212252621609099.
[I 2025-11-29 20:58:05,595] Trial 1 finished with value: 0.8247706162486352 and parameters: {'learning_rate': 0.024897120250139276, 'depth': 5, 'l2_leaf_reg': 1.9816588919583178, 'subsample': 0.9675433936565476, 'border_count': 95}. Best is trial 0 with value: 0.9212252621609099.
[I 2025-11-29 21:03:15,510] Trial 2 finished with value: 0.8574541559949392 and parameters: {'learning_rate': 0.04159845121979802, 'depth': 6, 'l2_leaf_reg': 1.8948022976618615, 'subsample': 0.7100551064410163, 'border_count': 85}. Best is trial 0 with value: 0.9212252621609099.
[I 2025-11-29 21:14:03,628] Trial 3 finished with value: 0.84901512538521 and parameters: {'learning_rate': 0.012572242546478072, 'depth': 9, 'l2_leaf_reg': 1.81

[INFO] Best CatBoost params: {'learning_rate': 0.18743013352582985, 'depth': 10, 'l2_leaf_reg': 6.852005346610192, 'subsample': 0.8440274016579897, 'border_count': 65}


In [16]:
evaluate_and_log(model_catboost_full,  X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

2025/11/30 01:08:11 INFO mlflow.tracking.fluent: Experiment with name 'Fraud-detection-custom-thresholds_v2' does not exist. Creating a new experiment.


[INFO] Logged metrics: {'roc_auc': np.float64(0.9764139849457184), 'pr_auc': np.float64(0.8693571706599657), 'precision': 0.6375523012552301, 'recall': 0.8848294217275586, 'f1': 0.7411085216333975, 'custom_loss': np.float64(0.4206234971382125)}
[INFO] Logged SHAP summary plot.


2025/11/30 01:09:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9764139849457184),
 'pr_auc': np.float64(0.8693571706599657),
 'precision': 0.6375523012552301,
 'recall': 0.8848294217275586,
 'f1': 0.7411085216333975,
 'custom_loss': np.float64(0.4206234971382125)}

In [17]:
evaluate_and_log(model_catboost_full,  X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9764139849457184), 'pr_auc': np.float64(0.8693571706599657), 'precision': 0.3744872045321352, 'recall': 0.9276554560851682, 'f1': 0.5335745598775311, 'custom_loss': np.float64(0.307379686388729)}
[INFO] Logged SHAP summary plot.


2025/11/30 01:10:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9764139849457184),
 'pr_auc': np.float64(0.8693571706599657),
 'precision': 0.3744872045321352,
 'recall': 0.9276554560851682,
 'f1': 0.5335745598775311,
 'custom_loss': np.float64(0.307379686388729)}

In [18]:
evaluate_and_log(model_catboost_full,  X_valid, y_valid, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.1)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9764139849457184), 'pr_auc': np.float64(0.8693571706599657), 'precision': 0.18166743436204513, 'recall': 0.9542705056859424, 'f1': 0.30522772123979414, 'custom_loss': np.float64(0.31044467775256546)}
[INFO] Logged SHAP summary plot.


2025/11/30 01:11:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9764139849457184),
 'pr_auc': np.float64(0.8693571706599657),
 'precision': 0.18166743436204513,
 'recall': 0.9542705056859424,
 'f1': 0.30522772123979414,
 'custom_loss': np.float64(0.31044467775256546)}

In [19]:
threshOpt, minScore = minimize_eval_metric_with_threshold(model_catboost_full,  X_valid, y_valid)
print(f"Optimal threshold: {threshOpt}, Minimum eval metric score: {minScore}")
evaluate_and_log(
    model_catboost_full, 
    X_valid, y_valid,
    experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", 
    hp_search_history=hist_df, hp_search_plots= plot_paths, 
    prediction_threshold=threshOpt)

Optimal threshold: 0.056640611442584844, Minimum eval metric score: 0.3448877298743523
[INFO] Logged metrics: {'roc_auc': np.float64(0.9764139849457184), 'pr_auc': np.float64(0.8693571706599657), 'precision': 0.12395263271805336, 'recall': 0.9699975804500363, 'f1': 0.2198157692729466, 'custom_loss': np.float64(0.3448877298743523)}
[INFO] Logged SHAP summary plot.


2025/11/30 01:12:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9764139849457184),
 'pr_auc': np.float64(0.8693571706599657),
 'precision': 0.12395263271805336,
 'recall': 0.9699975804500363,
 'f1': 0.2198157692729466,
 'custom_loss': np.float64(0.3448877298743523)}

## 2. Reduced Features

In [40]:
# ---- 1. Compute SHAP importance for each model ----
df_xgb = get_top_features_shap(model_xgboost_full, train, target).rename(
    columns={"importance": "importance_xgb"}
)
df_lgbm = get_top_features_shap(model_lgbm_full, train, target).rename(
    columns={"importance": "importance_lgbm"}
)
df_cat = get_top_features_shap(model_catboost_full, train, target).rename(
    columns={"importance": "importance_cat"}
)

# ---- 2. Merge all importance tables ----
df = (
    df_xgb.merge(df_lgbm, on="feature")
          .merge(df_cat, on="feature")
)

# ---- 3. Rank features within each model (1 = most important) ----
df["rank_xgb"] = df["importance_xgb"].rank(method="min", ascending=False)
df["rank_lgbm"] = df["importance_lgbm"].rank(method="min", ascending=False)
df["rank_cat"] = df["importance_cat"].rank(method="min", ascending=False)

# ---- 4. Determine the top 30% threshold ----
n_features = len(df)
top_30_cutoff = int(n_features * 0.30)

# ---- 5. Check if a feature appears in top 30% in each model ----
df["in_top_xgb"]  = df["rank_xgb"]  <= top_30_cutoff
df["in_top_lgbm"] = df["rank_lgbm"] <= top_30_cutoff
df["in_top_cat"]  = df["rank_cat"]  <= top_30_cutoff

# ---- 6. Apply the rule: “Top 30% for at least 2 models” ----
df["top_count"] = (
    df["in_top_xgb"].astype(int)
  + df["in_top_lgbm"].astype(int)
  + df["in_top_cat"].astype(int)
)

df_selected = df[df["top_count"] >= 2]  # final selected features

# ---- 7. Sort by mean importance (optional but nice) ----
df["importance_mean"] = df[["importance_xgb","importance_lgbm","importance_cat"]].mean(axis=1)
df_selected = df_selected.sort_values("importance_mean", ascending=False)

# ---- 8. Save to CSV ----
df.to_csv(f"{data_folder}/feature_importances_shap_fullmodels.csv", index=False)
df_selected.to_csv(f"{data_folder}/selected_features_shap.csv", index=False)

# ---- 9. List of selected feature names ----
selected_feature_list = df_selected["feature"].tolist()


NameError: name 'train' is not defined

In [ ]:
X_reduced = train[selected_feature_list]

In [ ]:
df.head()

In [ ]:
df_selected.head()

In [ ]:
del df_xgb, df_lgbm, df_cat, df_selected, df

In [ ]:
# X_reduced, feature_ranking = select_important_features(
#     train,
#     target,
#     top_n=100,                # or None to use cumulative 95%
#     use_shap=True,            # optional, slower but more accurate
#     remove_correlated=True,
#     correlation_threshold=0.95
# )

In [ ]:
X_train_reduced = X_train[selected_feature_list]
X_valid_reduced = X_valid[selected_feature_list]
X_tune_reduced = X_tune[selected_feature_list]


### 2.1. XGBoost 

In [ ]:
model_xgboost_reduced, best_params_xgboost_reduced, hist_df, plot_paths = train_xgb_optuna(X_train_reduced, y_train, X_tune_reduced, y_tune, X_valid_reduced, y_valid, n_trials=30)
evaluate_and_log(model_xgboost_reduced, X_valid_reduced, y_valid, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_xgboost_reduced, X_valid_reduced, y_valid, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

In [ ]:
threshOpt, minScore = minimize_eval_metric_with_threshold(model_xgboost_reduced, X_valid_reduced, y_valid)
print(f"Optimal threshold: {threshOpt}, Minimum eval metric score: {minScore}")
evaluate_and_log(
    model_xgboost_reduced, 
    X_valid_reduced, y_valid,
    experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", 
    hp_search_history=hist_df, hp_search_plots= plot_paths, 
    prediction_threshold=threshOpt)

### 2.2. LightGBM

In [ ]:
model_lgmb_reduced, best_params_lgmb_reduced, hist_df, plot_paths = train_lgbm_optuna(X_train_reduced, y_train, X_tune_reduced, y_tune, X_valid_reduced, y_valid, n_trials=30)
evaluate_and_log(model_lgmb_reduced, X_valid_reduced, y_valid, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_lgmb_reduced, X_valid_reduced, y_valid, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

In [ ]:
threshOpt, minScore = minimize_eval_metric_with_threshold(model_lgmb_reduced, X_valid_reduced, y_valid)
print(f"Optimal threshold: {threshOpt}, Minimum eval metric score: {minScore}")
evaluate_and_log(
    model_lgmb_reduced, 
    X_valid_reduced, y_valid,
    experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", 
    hp_search_history=hist_df, hp_search_plots= plot_paths, 
    prediction_threshold=threshOpt)

### 2.3. CatBoost

In [ ]:
model_catoost_reduced, best_params_catoost_reduced, hist_df, plot_paths = train_catboost_optuna(X_train_reduced, y_train, X_tune_reduced, y_tune, X_valid_reduced, y_valid, n_trials=30)
evaluate_and_log(model_catoost_reduced, X_valid_reduced, y_valid, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_catoost_reduced, X_valid_reduced, y_valid, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

In [ ]:
threshOpt, minScore = minimize_eval_metric_with_threshold(model_catoost_reduced, X_valid_reduced, y_valid)
print(f"Optimal threshold: {threshOpt}, Minimum eval metric score: {minScore}")
evaluate_and_log(
    model_catoost_reduced, 
    X_valid_reduced, y_valid,
    experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", 
    hp_search_history=hist_df, hp_search_plots= plot_paths, 
    prediction_threshold=threshOpt)

## 3. Ensemble

In [ ]:
base_models, X_tr, X_va, y_tr, y_va, X_test, y_test, best_params_dict = train_best_base_models_from_mlflow(
    train, target, test_size=0.15 ,experiment_name=EXPERIMENT_NAME
)

### 3.1. Logistic

In [ ]:
ensemble, base_models, X_meta_va, y_va, X_meta_test, y_test, best_params_log, hist_df, plot_paths = train_ensemble(
    base_models, X_tr, y_tr, X_va, y_va, X_test, y_test, n_trials=50
)

In [ ]:
evaluate_and_log(ensemble, pd.DataFrame(X_meta_va), y_va, experiment_name=EXPERIMENT_NAME, run_name="Ensemble_from_best_base_models", hp_search_history=hist_df)

### 3.2. NN

In [ ]:
nn_model, base_mdls, X_val, y_val, X_test, y_test, params, hist, plots = train_neural_network_ensemble(
    base_models=base_models,
    X_tr=X_tr,
    y_tr=y_tr,
    X_va=X_va,
    y_va=y_va,
    X_test=X_test,
    y_test=y_test,
    n_trials=30,
    epochs=100
)